In [30]:
import torch
from torch.nn import Embedding, LSTM
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

Let's create a dummy sequence, You can assume these are indexes of our words.
The objective of the whole process is to make a variable-length sequence suitable to be fed into an RNN (LSTM, GRU, etc) layer.
Here are the steps:

1. Create a variable-length sequence.
2. Pad the sequences to make them of the same length
3. Create an embedding for them.
4. Pack the embeddings (to speedup the RNN calculations)
5. Feed the (now packed) embeddings to LSTM to get outputs

To achieve the goal, we are going to use two utility functions from PyTorch. 

- **pad_sequence** (Simply adds zeros to the sequences so that they all have the same size)
- **pack_padded_sequence** ( Not necessarily required, but to be able to use the GPU more efficiently and speed up the RNN calculations) 

So the first methods pads (adds zeros) to the sequence, and the second method packs the previously padded sequence.

In [31]:
sequences = [   
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9,10]
]
sequences

[[1, 2, 3], [4, 5], [6, 7, 8, 9, 10]]

Before starting the padding, first we need to store the length of each sequence.
We need these lengths so that later on, we know exactly how to pack them and get rid of extra zeros in each sequence.
This way, we don't have to do additional calculations on some useless zeros(pad values) and this will speed up our RNN calculations.

In [32]:
sequence_lengths=torch.LongTensor([len(sequence) for sequence in sequences])

In [40]:
#embeding_dim = 6
#dictionary_size = 11

embedding = Embedding(num_embeddings=11, embedding_dim=6)
lstm = LSTM(input_size=6, hidden_size=2, batch_first=True) 

In [34]:
#Padding
sequences=[torch.LongTensor(sequence) for sequence in sequences]

sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
sequences_padded

tensor([[ 1,  2,  3,  0,  0],
        [ 4,  5,  0,  0,  0],
        [ 6,  7,  8,  9, 10]])

In [35]:
#Embedding
sequences_embedded=embedding(sequences_padded)
sequences_embedded

tensor([[[ 1.1744, -0.8434,  0.1616,  0.3518, -1.0929, -0.3832],
         [-0.2835, -0.0850, -1.0739, -0.6135, -1.1897,  0.8438],
         [ 0.1001,  1.1455, -1.1408, -1.1769,  0.4091,  1.0318],
         [-0.6493,  1.5232,  1.6697,  0.2102,  1.1498,  0.6828],
         [-0.6493,  1.5232,  1.6697,  0.2102,  1.1498,  0.6828]],

        [[ 2.4038, -0.6158,  1.0027, -2.2223, -0.8160, -0.0100],
         [-0.7905, -0.6819,  0.7061, -0.4486,  1.1292,  0.3120],
         [-0.6493,  1.5232,  1.6697,  0.2102,  1.1498,  0.6828],
         [-0.6493,  1.5232,  1.6697,  0.2102,  1.1498,  0.6828],
         [-0.6493,  1.5232,  1.6697,  0.2102,  1.1498,  0.6828]],

        [[ 2.0162,  0.5278,  0.0818,  1.3061, -0.1523, -0.8143],
         [-0.8136, -0.5434, -0.5993,  1.3536, -0.7848, -0.4640],
         [-0.9396,  0.4093,  0.4099,  0.2460,  0.4102,  1.4678],
         [-1.1875, -0.1017, -1.1088, -0.6476, -0.3442, -1.7732],
         [-1.4413, -0.3622, -1.8406,  0.2716, -0.7255, -1.1358]]],
       grad_fn=<Emb

In [36]:
#Packing
sequences_packed = pack_padded_sequence(sequences_embedded, sequence_lengths.cpu().numpy(), batch_first=True,enforce_sorted=False)
sequences_packed

PackedSequence(data=tensor([[ 2.0162,  0.5278,  0.0818,  1.3061, -0.1523, -0.8143],
        [ 1.1744, -0.8434,  0.1616,  0.3518, -1.0929, -0.3832],
        [ 2.4038, -0.6158,  1.0027, -2.2223, -0.8160, -0.0100],
        [-0.8136, -0.5434, -0.5993,  1.3536, -0.7848, -0.4640],
        [-0.2835, -0.0850, -1.0739, -0.6135, -1.1897,  0.8438],
        [-0.7905, -0.6819,  0.7061, -0.4486,  1.1292,  0.3120],
        [-0.9396,  0.4093,  0.4099,  0.2460,  0.4102,  1.4678],
        [ 0.1001,  1.1455, -1.1408, -1.1769,  0.4091,  1.0318],
        [-1.1875, -0.1017, -1.1088, -0.6476, -0.3442, -1.7732],
        [-1.4413, -0.3622, -1.8406,  0.2716, -0.7255, -1.1358]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

In [37]:
#LSTM
output_packed, (hidden,context)=lstm(sequences_packed)
output_packed

PackedSequence(data=tensor([[ 0.1268, -0.0941],
        [-0.0288, -0.1231],
        [-0.0624,  0.0266],
        [ 0.0636, -0.1958],
        [ 0.0178, -0.1081],
        [-0.0291,  0.0587],
        [ 0.0923, -0.1137],
        [ 0.0393,  0.0526],
        [ 0.0339, -0.1109],
        [ 0.0286, -0.1766]], grad_fn=<CatBackward>), batch_sizes=tensor([3, 3, 2, 1, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

LSTM returns a packed output since it has received a packed input. The *packed_output* is in fact is a Named Tuple which provides some additional information that we might not care about. The actual output that we want is in compress(packed) form in the *data* field. How should we uncompress it and get our actual output values? 


**UnPacking**

*pad_packed_sequence*  might seems a bit confusing at the beginning but its role is actually very simple. Whenever we pack something we need to be able to unpack it again, right? (Think of zip and unzip). So here, this function just un-packs a sequence. (Which obviously should have already been packed).

In [38]:
unpacked_out, input_sequence_sizes = pad_packed_sequence(output_packed, batch_first=True)
unpacked_out

tensor([[[-0.0288, -0.1231],
         [ 0.0178, -0.1081],
         [ 0.0393,  0.0526],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.0624,  0.0266],
         [-0.0291,  0.0587],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.1268, -0.0941],
         [ 0.0636, -0.1958],
         [ 0.0923, -0.1137],
         [ 0.0339, -0.1109],
         [ 0.0286, -0.1766]]], grad_fn=<IndexSelectBackward>)

If we closely look at this unpacked output and compare it to the packed version above, we can see that the packed version does not have any results for the padded rows (i.e rows with zero, remember we used 0 for padding) . This  means that we have not done any unnecessary computations in the LSTM. Thats the benefit of using a packed input for our recurrent layer.

Additionally, if we need the *hidden state* of the last layer of LSTM (context vector), we can get it like this:

In [39]:
hidden[-1:]

tensor([[[ 0.0393,  0.0526],
         [-0.0291,  0.0587],
         [ 0.0286, -0.1766]]], grad_fn=<SliceBackward>)